In [ ]:
import redshift_connector
from datetime import timedelta,datetime
import numpy as np
import pandas as pd 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import re

# Connects to Redshift cluster using AWS credentials
username = input("Enter the user name :  ")
password = input("Enter the password :  ")

conn = redshift_connector.connect(
    host='10.19.2.81',
    database='zeta_reports',
    user=username,
    password=password,
 )

cursor: redshift_connector.Cursor = conn.cursor()

In [ ]:
cursor.execute("select * from temp.user_stats_sj5")
rfm_table1: pd.DataFrame = cursor.fetch_dataframe()
rfm_table1.head()

In [ ]:
#Finding Optimum Clusters
sse={}
tx_recency = rfm_table1[['recency']]
for k in range(1, 5):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_recency)
    tx_recency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
#### Function for ordering clusters
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    new_cluster_field_name = 'new_'+cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name, ascending = ascending).reset_index(drop=True)
    df_new['index']=df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, 'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [ ]:
#Recency Clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(rfm_table1[['recency']])
rfm_table1['RecencyCluster']=kmeans.predict(rfm_table1[['recency']])
rfm_table1 = order_cluster('RecencyCluster','recency', rfm_table1, False)

In [ ]:
rfm_table1.head()

In [ ]:
#Frequency Clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(rfm_table1[['frequency']])
rfm_table1['FrequencyCluster']=kmeans.predict(rfm_table1[['frequency']])
rfm_table1 = order_cluster('FrequencyCluster','frequency', rfm_table1, True)


In [ ]:
rfm_table1.head()

In [ ]:
#Monetary Clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(rfm_table1[['monetary']])
rfm_table1['MonetaryCluster']=kmeans.predict(rfm_table1[['monetary']])
rfm_table1 = order_cluster('MonetaryCluster','monetary', rfm_table1, True)

In [ ]:
rfm_table1.head()

In [ ]:
# Define a function to map the values 
def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 

In [ ]:
# Creating Recency Custer Tag Column
c=rfm_table1.groupby('RecencyCluster')['recency'].agg(['min','max','mean']).reset_index()
r1='<='+str(c.iloc[2,2]) 
r2=str(c.iloc[1,1])+' to '+str(c.iloc[1,2]) 
r3='>='+str(c.iloc[0,1])
# Create the dictionary 
R ={0 : r1, 1 : r2, 2 : r3} 
rfm_table1['RecencyClusterTag']=rfm_table1['RecencyCluster'].apply(set_value, args =(R, )) 

In [ ]:
# Creating Frequency Custer Tag Column
a=rfm_table1.groupby('FrequencyCluster')['frequency'].agg(['min','max','mean']).reset_index()
f1='<='+str(a.iloc[0,2]) 
f2=str(a.iloc[1,1])+' to '+str(a.iloc[1,2]) 
f3='>='+str(a.iloc[2,1])
# Create the dictionary 
F ={0 : f1, 1 : f2, 2 :f3} 
rfm_table1['FrequencyClusterTag']=rfm_table1['FrequencyCluster'].apply(set_value, args =(F, )) 

In [ ]:
# Creating Monetary Custer Tag Column
b=rfm_table1.groupby('MonetaryCluster')['monetary'].agg(['min','max','mean']).reset_index()
m1='<='+str(int(round(b.iloc[0,2]))) 
m2=str(int(round(b.iloc[0,2])+1))+' to '+str(int(round(b.iloc[1,2]))) 
m3='>='+str(int(round(b.iloc[1,2])+1))
# Create the dictionary 
M ={0 : m1, 1 : m2, 2 :m3} 
rfm_table1['MonetaryClusterTag']=rfm_table1['MonetaryCluster'].apply(set_value, args =(M, )) 

In [ ]:
rfm_table1["FrequencyClusterTag"] = rfm_table1["FrequencyClusterTag"].astype(pd.api.types.CategoricalDtype(categories=[f1, f2, f3]))
rfm_table1["MonetaryClusterTag"] = rfm_table1["MonetaryClusterTag"].astype(pd.api.types.CategoricalDtype(categories=[m1, m2,m3]))
rfm_table1["RecencyClusterTag"] = rfm_table1["RecencyClusterTag"].astype(pd.api.types.CategoricalDtype(categories=[r1, r2,r3]))

In [ ]:
rfm_table1.head()

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(8, 8), dpi=80)

rfm_table1["RecencyClusterTag"].value_counts().plot(kind='bar',xlabel='Cluster Range',ylabel='No. of. Customers',title='Recency Cluster Distribution').figure.savefig('recency.jpeg')

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(10, 10), dpi=80)

rfm_table1["FrequencyClusterTag"].value_counts().plot(kind='bar',xlabel='Cluster Range',ylabel='No. of. Customers',title='Frequency Cluster Distribution').figure.savefig('Frequency.jpeg')

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(15, 15), dpi=80)
plt.rcParams.update({'font.size': 22})
rfm_table1["MonetaryClusterTag"].value_counts().plot(kind='bar',xlabel='Cluster Range',ylabel='No. of. Customers',title='Monetary Cluster Distribution').figure.savefig('Monetary.jpeg')

In [ ]:
rfm_table1["combinedrfm"]=rfm_table1["RecencyCluster"].astype(str)+rfm_table1["FrequencyCluster"].astype(str)+rfm_table1["MonetaryCluster"].astype(str)

In [ ]:
def flag_df(df):
    if (df['combinedrfm'] in '222'):
        return 'Best Customers'
    elif re.findall(".2.",df['combinedrfm']) or re.findall("211",df['combinedrfm']):
        return 'Loyal Customers'
    elif re.findall("..2",df['combinedrfm']):
        return 'Whales Customers'
    elif re.findall("00.",df['combinedrfm']):
        return 'Lost Customers'
    elif re.findall("10.",df['combinedrfm']) or re.findall("01.",df['combinedrfm']) or (df['combinedrfm'] in '200') or (df['combinedrfm'] in '210'):
        return 'Slipping Customers'
    elif (df['combinedrfm'] in '11.') or (df['combinedrfm'] in '201'):
        return 'Potential Customers'
    else : 
        return 'Regular'

In [ ]:
rfm_table1['Flag'] = rfm_table1.apply(flag_df, axis = 1)

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(8, 8), dpi=80)

rfm_table1["Flag"].value_counts().plot(kind='pie',autopct='%1.0f%%',title='Distribution of Customer Segments').figure.savefig('Customer Segmenr.jpeg')

In [ ]:
with conn.cursor() as cursor:
    cursor.write_dataframe(rfm_table1, "temp.hacksjfinal3")
    conn.commit()